# Overview of `ucompress` models

This notebook will:

* Explain what a `ucompress` model is and how to build them
* List the pre-defined models
* Explain how to non-dimensionalise a parameter set

## What is a `ucompress` model?

In `ucompress`, a model is a Python object that contains information about the material to be simulated.  Currently, there are templates for two types of materials.  These are 

* **Poroelastic** - a class for poroelastic materials
* **Hydrogel** - a class for hydrogels

The difference between the two is that the hydrogel model accounts for osmotic effects.  A poroelastic model requires three components

* **Mechanics** - describes the mechanical model the material
* **Permeability** - describes the permeability of the material
* **Parameters** - describes the material parameters

A hydrogel model requires the same three components as a poroelastic model (see above), plus

* **Osmosis** - describes the osmotic pressure

From a programming perspective, a `ucompress` model has two main components.  The first is a symbolic, or mathematically exact, representation of the expressions used in the model, such as the stress-strain relations, permeability, and their derivatives.  The second component of the model is NumPy functions that have been created from the symbolic expressions, which are then used when computing numerical solutions of the model.  The rationale behind this approach is discussed in Tutorial 04.

## Summary of implemented models

The following models are implemented in `ucompress`:

### Mechanics models

* **NeoHookean** - a two-parameter neo-Hookean material
* **FibreReinforced** - a neo-Hookean matrix, reinforced by a transversely isotropic fibre network
* **FibreRecruitment** - a neo-Hookean matrix, reinforced by a tranversely isotropic fibre network that is not fully engaged (i.e. there is some slack in the initial network).

The `FibreRecruitment` class has different models to describe how the fibre network tautens as it deforms.


### Permeability models

* **Constant** - a constant permeability
* **KozenyCarman** - the classical Kozeny-Carmen permeability
* **HolmesMow** - a permeability function proposed by Holmes & Mow

### Osmosis models

* **FloryHuggins** - the osmotic pressure derived from Flory-Huggins theory

## Parameters

Several pre-defined parameter sets are included with `ucompress`:

* **NeoHookean** - a parameter set for a poroelastic material with neo-Hookean elasticity
* **Hydrogel** - a parameter set for a hydrogel with neo-Hookean elasticity and Flory-Huggins osmotic pressure
* **FibreReinforced** - a parameter set for a fibre-reinforced neo-Hookean material
* **FibreRecruitment** - a parameter set for a fibre-reinforced neo-Hookean material with slack in the fibre network

Parameter sets are stored as objects from the `Parameter` class, which splits the parameter set into two Python dictionaries.  The first dictionary is an attribute called `physical` that contains all of the physical parameters associated with the material or load.  The second dictionary is an attribute called `computational` that contains parameter sets associated with the simulation (e.g. space and time points).

The parameter values can be printed to the screen using Python's `print` function

In [1]:
import ucompress as uc

# Define the parameters
pars = uc.parameters.example_parameters.NeoHookean()

# Print values (in SI units)
print(pars)

Dimensional parameter values
---------------------------------------
R = 5.00e-03
E_m = 5.00e+04
nu_m = 0.00e+00
k_0 = 2.00e-13
phi_0 = 8.00e-01
lam_z = 5.00e-01
beta_r = 1.00e+00
beta_z = 1.00e+00
F = -1.00e+00
t_start = 1.00e-01
t_end = 1.00e+04

Computational parameter values
-----------------------------------------
N = 40
Nt = 200
t_spacing = log



From this set, we see that the initial radius of the material is $R = 5$ mm, the Young's modulus is $E_m = 50$ kPa, the Poisson's ratio is $\nu_m = 0$, tinitial permeability is $k_0 = 2 \times 10^{-7}$ mm/Pa/s, the initial porosity is $\phi_0 = 0.8$.  

The axial strech is fixed at $\lambda_z = 0.5$, corresponding to an imposed axial strain of $\epsilon_z = 1 - \lambda_z = 0.5$.  The value of the applied force is $F = -1$ N.  For displacement-controlled experiments in which $\lambda_z$ is fixed, the value of the force $F$ defined in the parameters is not used.  For force-controlled experiments, the force is determined by $F$ and the axial stretch $\lambda_z$ serves as an initial guess for the solution.

The quantities $\beta_r$ and $\beta_z$ are pre-stretches; these are mainly used for hydrogel models to account for stretches due to hydration.  Setting their values to one means the matrix has not been pre-stretched so the elastic stress in the reference state is zero.

The parameters $t_\text{start}$ and $t_\text{end}$ describe the start and end time of the experiment.  A non-zero value of $t_\text{start}$ is needed because of the choice to use logarithmic spacing of the time variable.

The computational parameters define the number of spatial grid point $N$, the number of time steps $N_t$, and the spacing of time points (linear or log).

### Accessing parameter values

The `physical` and `computational` dictionaries are attributes of Parameter objects and can be accessed using dot notation.  For example, to access and then print the dictionary of physical parameter values, the following command can be used:

In [2]:
# print the dictionary of physical parameter values
print(pars.physical)

{'R': 0.005, 'E_m': 50000.0, 'nu_m': 0, 'k_0': 2e-13, 'phi_0': 0.8, 'lam_z': 0.5, 'beta_r': 1, 'beta_z': 1, 'F': -1, 't_start': 0.1, 't_end': 10000.0}


Individual parameters can then be accessed using the dictionary keys:

In [3]:
# print the Poisson's ratio
print(pars.physical['nu_m'])

0


**Warning**: although parameter values can also be changed by directly editing the attributes/dictonaries of Parameter objects, the best way to change a parameter value is to use the `update` method, as explained in Tutorial 04.

### Non-dimensionalisation of parameters

`ucompress` contains functionality for generating a non-dimensional parameter set from dimensional parameters.  Using non-dimensional parameters can improve the performance and speed of solvers.  

All of the example parameter sets can be non-dimensionalised by passing setting the `nondim` argument to `True` when creating the parameter set:

In [4]:
nondim_pars = uc.parameters.example_parameters.NeoHookean(nondim = True)
print(nondim_pars)

Dimensional parameter values
---------------------------------------
R = 5.00e-03
E_m = 5.00e+04
nu_m = 0.00e+00
k_0 = 2.00e-13
phi_0 = 8.00e-01
lam_z = 5.00e-01
beta_r = 1.00e+00
beta_z = 1.00e+00
F = -1.00e+00
t_start = 1.00e-01
t_end = 1.00e+04

Non-dimensional parameter values
-----------------------------------------
R = 1.00e+00
E_m = 1.00e+00
nu_m = 0.00e+00
k_0 = 1.00e+00
phi_0 = 8.00e-01
lam_z = 5.00e-01
beta_r = 1.00e+00
beta_z = 1.00e+00
F = -8.00e-01
t_start = 4.00e-05
t_end = 4.00e+00

Computational parameter values
-----------------------------------------
N = 40
Nt = 200
t_spacing = log



When non-dimensionalising a parameter set, a third attribute to the Parameter object will be created in order to store the original dimensional physical variables.  This new attribute is called `dimensional`.  The `physical` attribute is overwritten with the new non-dimensional physical varibles.

In [5]:
# print the new dimensional attribute
print(nondim_pars.dimensional)

# print the physical attribute
print(nondim_pars.physical)

{'R': 0.005, 'E_m': 50000.0, 'nu_m': 0, 'k_0': 2e-13, 'phi_0': 0.8, 'lam_z': 0.5, 'beta_r': 1, 'beta_z': 1, 'F': -1, 't_start': 0.1, 't_end': 10000.0}
{'R': 1.0, 'E_m': 1.0, 'nu_m': 0, 'k_0': 1.0, 'phi_0': 0.8, 'lam_z': 0.5, 'beta_r': 1, 'beta_z': 1, 'F': -0.8, 't_start': 4e-05, 't_end': 4.0}


## Examples

In the first example, a model is defined for a poroelastic material with a neo-Hookean response.  An example parameter set for neo-Hookean poroelastic materials has been provided.  The Kozeny-Carmen permeability law will be used.

In [6]:
import ucompress as uc

# Define the parameters
pars = uc.parameters.example_parameters.NeoHookean()

# Define the mechanics
mech = uc.mechanics.NeoHookean()

# Define the permeability
perm = uc.permeability.KozenyCarman()

# Build the model
model = uc.base_models.Poroelastic(mechanics = mech,
                                   permeability = perm,
                                   parameters = pars)

In the second example, we define a model for a fibre-reinforced hydrogel.  The fibre network is assumed to be fully engaged.  An example parameter set for this type of material has been provided too.

In [7]:
# Define the parameters
pars = uc.parameters.example_parameters.FibreReinforced()

# Define the mechanics
mech = uc.mechanics.FibreReinforced()

# Define the permeability
perm = uc.permeability.HolmesMow()

# Define the osmotic model
os = uc.osmosis.FloryHuggins()

# Build the model
model = uc.base_models.Hydrogel(mechanics = mech,
                                permeability = perm,
                                osmosis = os,
                                parameters = pars)